## 1. Import and install dependencies

In [58]:
#pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe==0.8.6.2 sklearn matplotlib --user

SyntaxError: invalid syntax (<ipython-input-58-b5defd71d18c>, line 2)

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2. Keypoints using Mediapipe Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

## 3. Extract Keypoints Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## 4. Setup Folders for Collection

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['ola', 'boa noite', 'eu_te_amo'])

# Thirty videos worth of data
no_sequences = 65

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoints Values for Training and Testing

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## 7. Build and Train LSTM Neural Network

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [22]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
6/6 [==============================] - 5s 187ms/step - loss: 1.7246 - categorical_accuracy: 0.3460
Epoch 2/1000
6/6 [==============================] - 1s 104ms/step - loss: 1.2409 - categorical_accuracy: 0.2846
Epoch 3/1000
6/6 [==============================] - 1s 115ms/step - loss: 1.2625 - categorical_accuracy: 0.3117
Epoch 4/1000
6/6 [==============================] - 1s 97ms/step - loss: 1.0828 - categorical_accuracy: 0.3500
Epoch 5/1000
6/6 [==============================] - 1s 89ms/step - loss: 1.1011 - categorical_accuracy: 0.3671
Epoch 6/1000
6/6 [==============================] - 1s 84ms/step - loss: 1.0864 - categorical_accuracy: 0.4057
Epoch 7/1000
6/6 [==============================] - 1s 87ms/step - loss: 1.0969 - categorical_accuracy: 0.4028
Epoch 8/1000
6/6 [==============================] - ETA: 0s - loss: 1.0660 - categorical_accuracy: 0.36 - 1s 88ms/step - loss: 1.0659 - categorical_accuracy: 0.3661
Epoch 9/1000
6/6 [==============================] - 1s 

6/6 [==============================] - 1s 127ms/step - loss: 0.0649 - categorical_accuracy: 0.9741
Epoch 145/1000
6/6 [==============================] - 1s 91ms/step - loss: 0.0631 - categorical_accuracy: 0.9759
Epoch 146/1000
6/6 [==============================] - 1s 92ms/step - loss: 0.0821 - categorical_accuracy: 0.9636
Epoch 147/1000
6/6 [==============================] - 1s 84ms/step - loss: 0.0446 - categorical_accuracy: 0.9839
Epoch 148/1000
6/6 [==============================] - 1s 98ms/step - loss: 0.0547 - categorical_accuracy: 0.9797
Epoch 149/1000
6/6 [==============================] - 1s 103ms/step - loss: 0.0405 - categorical_accuracy: 0.9791
Epoch 150/1000
6/6 [==============================] - 1s 94ms/step - loss: 0.0584 - categorical_accuracy: 0.9800
Epoch 151/1000
6/6 [==============================] - 1s 111ms/step - loss: 0.0635 - categorical_accuracy: 0.9695
Epoch 152/1000
6/6 [==============================] - 1s 103ms/step - loss: 0.0553 - categorical_accuracy: 0

6/6 [==============================] - 0s 80ms/step - loss: 0.0435 - categorical_accuracy: 0.9779
Epoch 288/1000
6/6 [==============================] - 0s 80ms/step - loss: 0.0352 - categorical_accuracy: 0.9843
Epoch 289/1000
6/6 [==============================] - 0s 83ms/step - loss: 0.0478 - categorical_accuracy: 0.9893
Epoch 290/1000
6/6 [==============================] - 0s 80ms/step - loss: 0.0278 - categorical_accuracy: 0.9925
Epoch 291/1000
6/6 [==============================] - 0s 81ms/step - loss: 0.0317 - categorical_accuracy: 0.9811
Epoch 292/1000
6/6 [==============================] - 0s 79ms/step - loss: 0.0258 - categorical_accuracy: 0.9914
Epoch 293/1000
6/6 [==============================] - 0s 83ms/step - loss: 0.0634 - categorical_accuracy: 0.9612
Epoch 294/1000
6/6 [==============================] - 0s 80ms/step - loss: 0.0122 - categorical_accuracy: 0.9969
Epoch 295/1000
6/6 [==============================] - 0s 81ms/step - loss: 0.0121 - categorical_accuracy: 0.998

6/6 [==============================] - 0s 81ms/step - loss: 2.9063e-05 - categorical_accuracy: 1.0000
Epoch 429/1000
6/6 [==============================] - 0s 80ms/step - loss: 2.8932e-05 - categorical_accuracy: 1.0000
Epoch 430/1000
6/6 [==============================] - 0s 78ms/step - loss: 1.5250e-05 - categorical_accuracy: 1.0000
Epoch 431/1000
6/6 [==============================] - 0s 79ms/step - loss: 3.1293e-05 - categorical_accuracy: 1.0000
Epoch 432/1000
6/6 [==============================] - 0s 81ms/step - loss: 3.5579e-05 - categorical_accuracy: 1.0000
Epoch 433/1000
6/6 [==============================] - 0s 80ms/step - loss: 3.1139e-05 - categorical_accuracy: 1.0000
Epoch 434/1000
6/6 [==============================] - 0s 80ms/step - loss: 2.1677e-05 - categorical_accuracy: 1.0000
Epoch 435/1000
6/6 [==============================] - 1s 87ms/step - loss: 2.7466e-05 - categorical_accuracy: 1.0000
Epoch 436/1000
6/6 [==============================] - 0s 79ms/step - loss: 3.68

6/6 [==============================] - 0s 80ms/step - loss: 7.7205e-06 - categorical_accuracy: 1.0000
Epoch 569/1000
6/6 [==============================] - 0s 81ms/step - loss: 8.4000e-06 - categorical_accuracy: 1.0000
Epoch 570/1000
6/6 [==============================] - 0s 80ms/step - loss: 5.2032e-06 - categorical_accuracy: 1.0000
Epoch 571/1000
6/6 [==============================] - 0s 81ms/step - loss: 5.1470e-06 - categorical_accuracy: 1.0000
Epoch 572/1000
6/6 [==============================] - 0s 80ms/step - loss: 3.6626e-06 - categorical_accuracy: 1.0000
Epoch 573/1000
6/6 [==============================] - 0s 82ms/step - loss: 5.1547e-06 - categorical_accuracy: 1.0000
Epoch 574/1000
6/6 [==============================] - 0s 80ms/step - loss: 7.6146e-06 - categorical_accuracy: 1.0000
Epoch 575/1000
6/6 [==============================] - 0s 81ms/step - loss: 6.4779e-06 - categorical_accuracy: 1.0000
Epoch 576/1000
6/6 [==============================] - 0s 81ms/step - loss: 5.55

6/6 [==============================] - 0s 80ms/step - loss: 1.6790e-06 - categorical_accuracy: 1.0000
Epoch 709/1000
6/6 [==============================] - 0s 80ms/step - loss: 1.4950e-06 - categorical_accuracy: 1.0000
Epoch 710/1000
6/6 [==============================] - 0s 83ms/step - loss: 1.8848e-06 - categorical_accuracy: 1.0000
Epoch 711/1000
6/6 [==============================] - 0s 79ms/step - loss: 1.8420e-06 - categorical_accuracy: 1.0000
Epoch 712/1000
6/6 [==============================] - 0s 80ms/step - loss: 1.4398e-06 - categorical_accuracy: 1.0000
Epoch 713/1000
6/6 [==============================] - 0s 79ms/step - loss: 2.2646e-06 - categorical_accuracy: 1.0000
Epoch 714/1000
6/6 [==============================] - 0s 82ms/step - loss: 2.0883e-06 - categorical_accuracy: 1.0000
Epoch 715/1000
6/6 [==============================] - 0s 79ms/step - loss: 2.3936e-06 - categorical_accuracy: 1.0000
Epoch 716/1000
6/6 [==============================] - 0s 79ms/step - loss: 2.69

6/6 [==============================] - 0s 79ms/step - loss: 6.6294e-07 - categorical_accuracy: 1.0000
Epoch 849/1000
6/6 [==============================] - 0s 82ms/step - loss: 8.7150e-07 - categorical_accuracy: 1.0000
Epoch 850/1000
6/6 [==============================] - 0s 79ms/step - loss: 1.0073e-06 - categorical_accuracy: 1.0000
Epoch 851/1000
6/6 [==============================] - 0s 81ms/step - loss: 1.0011e-06 - categorical_accuracy: 1.0000
Epoch 852/1000
6/6 [==============================] - 0s 81ms/step - loss: 7.7725e-07 - categorical_accuracy: 1.0000
Epoch 853/1000
6/6 [==============================] - 1s 84ms/step - loss: 9.7825e-07 - categorical_accuracy: 1.0000
Epoch 854/1000
6/6 [==============================] - 0s 81ms/step - loss: 9.4502e-07 - categorical_accuracy: 1.0000
Epoch 855/1000
6/6 [==============================] - 0s 81ms/step - loss: 1.1856e-06 - categorical_accuracy: 1.0000
Epoch 856/1000
6/6 [==============================] - 0s 81ms/step - loss: 8.74

6/6 [==============================] - 0s 80ms/step - loss: 4.3035e-07 - categorical_accuracy: 1.0000
Epoch 989/1000
6/6 [==============================] - 0s 81ms/step - loss: 3.5513e-07 - categorical_accuracy: 1.0000
Epoch 990/1000
6/6 [==============================] - 0s 80ms/step - loss: 3.1621e-07 - categorical_accuracy: 1.0000
Epoch 991/1000
6/6 [==============================] - 0s 81ms/step - loss: 3.0895e-07 - categorical_accuracy: 1.0000
Epoch 992/1000
6/6 [==============================] - 0s 79ms/step - loss: 2.9725e-07 - categorical_accuracy: 1.0000
Epoch 993/1000
6/6 [==============================] - 0s 80ms/step - loss: 3.1753e-07 - categorical_accuracy: 1.0000
Epoch 994/1000
6/6 [==============================] - 0s 81ms/step - loss: 3.7977e-07 - categorical_accuracy: 1.0000
Epoch 995/1000
6/6 [==============================] - 0s 82ms/step - loss: 2.6617e-07 - categorical_accuracy: 1.0000
Epoch 996/1000
6/6 [==============================] - 0s 80ms/step - loss: 3.83

In [1]:
model.summary()

NameError: name 'model' is not defined

## 9. Save Weights

In [23]:
model.save('action.h5')

In [19]:
model.load_weights('action.h5')

## 11. Test in Real Time¶


In [20]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.py

<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te

<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te

...

ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.py

eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<cl

<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python